# REDES NEURAIS ARTIFICIAIS - AVALIAÇÃO, MELHORIA E AFINAÇÃO

[Aula 5 de Deep Learning](http://bit.ly/dn-unb05) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

## Importar as libs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

## Importar o Dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

## Transformar dados categóricos 

In [3]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 - Vamos construir uma ANN!

## Importar as libs e pacotes do Keras 

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

## Inicializar a ANN

In [8]:
ann = tf.keras.models.Sequential()

## Adicionar a camada de entrada e 1ª camada oculta

In [9]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

## Adicionar 2ª camada oculta

In [10]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'softplus'))

## Adicionar camada de saída

In [11]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'tanh'))

# Parte 3 - Treinando a ANN

## Compilar a rede neural

In [12]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Treinando a ANN com o conjunto de treinamento

In [ ]:
ann.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 3.2290 - accuracy: 0.7907
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 3.1540 - accuracy: 0.7955
Epoch 3/100
800/800 [==============================] - 1s 875us/step - loss: 3.1826 - accuracy: 0.7937
Epoch 4/100
147/800 [====>.........................] - ETA: 0s - loss: 2.9590 - accuracy: 0.8082

# Parte 4 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de teste

In [ ]:
y_pred = ann.predict(X_test)
y_pred

In [ ]:
y_pred = (y_pred > 0.5)
y_pred

## Criando uma Matriz de Confusão

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Parte 5 - Avaliação, Melhoria e Afinação

## Avaliar a ANN

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

## Melhorar a ANN

In [ ]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

## Afinar a ANN

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [10, 25],
              'epochs': [50, 100],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
mean_score = grid_search.cv_results_['mean_test_score']
std_score = grid_search.cv_results_['std_test_score']
params_score = grid_search.cv_results_['params']

print('Melhores parametros {}\nMelhor Acuracia: {}'.format(best_parameters, best_accuracy))
print('Média:        Var:      Param:')
for mean, stdev, param in zip(mean_score, std_score, params_score):
    print("%f (%f) com: %r" % (mean, stdev, param))